In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import gc
import time
from sklearn.preprocessing import scale
from sklearn.preprocessing import PolynomialFeatures

/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print('loading files...')
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
sample_submission = pd.read_csv('sample_submission.csv')

loading files...


In [3]:
train.head()

,connection_id,cont_1,cont_2,cont_3,cont_4,cont_5,cont_6,cont_7,cont_8,cont_9,...,cat_15,cat_16,cat_17,cat_18,cat_19,cat_20,cat_21,cat_22,cat_23,target
0,cxcon_1,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,2
1,cxcon_4,0,520,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
2,cxcon_7,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
3,cxcon_10,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
4,cxcon_13,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,2


In [4]:
test.head()

,connection_id,cont_1,cont_2,cont_3,cont_4,cont_5,cont_6,cont_7,cont_8,cont_9,...,cat_14,cat_15,cat_16,cat_17,cat_18,cat_19,cat_20,cat_21,cat_22,cat_23
0,cxcon_2,0,331,6085,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,13,13,255,255
1,cxcon_5,0,520,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,511,511,255,255
2,cxcon_8,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,511,511,255,255
3,cxcon_11,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,510,509,255,255
4,cxcon_14,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,511,511,255,255


In [5]:
sample_submission.head()

,connection_id,target
0,cxcon_2,2
1,cxcon_5,0
2,cxcon_8,0
3,cxcon_11,1
4,cxcon_14,2


In [6]:
target = train['target']
test_ID = test['connection_id']
train.drop(['target','connection_id'] , axis=1 , inplace = True)
test.drop(['connection_id'] , axis=1 , inplace = True)
tr_ID = train.shape[0]

data = pd.concat([train, test] , axis = 0)

In [7]:
cont = data.columns[data.columns.str.startswith('cont')]
cat = data.columns[data.columns.str.startswith('cat')]

In [8]:
for col in cont :
    data[col] = scale(data[col].values)

/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [9]:
for col in tqdm(cat) :
    data[col] = data[col].astype('category')
    data[col] = data[col].cat.codes

100%|██████████| 23/23 [00:00<00:00, 45.94it/s]


In [10]:
poly = PolynomialFeatures(degree = 2 ,interaction_only = True)
feat_new = poly.fit_transform(data[cont].values)
data = np.concatenate([data.values , feat_new] , axis = 1)
data = pd.DataFrame(data)

In [11]:
data['n0'] = (data==0).sum(axis=1)
data['n1'] = (data==1).sum(axis=1)
data['n2'] = (data==2).sum(axis=1)
data['n3'] = (data==3).sum(axis=1)
data['n4'] = (data==4).sum(axis=1)
data['n5'] = (data==5).sum(axis=1)
data['n6'] = (data==6).sum(axis=1)

In [ ]:
train = data.iloc[:tr_ID]
test = data.iloc[tr_ID:]

In [ ]:
# lgb

t1 = time.time()

X = train
X = X.values
y = target.values
sub = np.zeros([test.shape[0],3])


feat_imp = []
nrounds = 5000 
kfold = 5

params = { 'metric':'multi_logloss','learning_rate' : 0.01, 'max_depth':10, 'max_bin':15,  'objective': 'multiclass', 
          'num_class':3, 'feature_fraction': 0.8,'bagging_fraction':0.9,'bagging_freq':10,  'min_data': 500}

skf = StratifiedKFold(n_splits=kfold, random_state=1)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' lgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]
    lgb_model = lgb.train(params, lgb.Dataset(X_train, label=y_train), nrounds, lgb.Dataset(X_eval, label=y_eval), 
                          verbose_eval=10, early_stopping_rounds = 100)
   
    l = lgb_model.predict(test.values, num_iteration=lgb_model.best_iteration)
    
    sub += l/(kfold)
    feat_imp.append(lgb_model.feature_importance())


    
t2 = time.time()
print "Time taken {} hr".format((t2-t1)/(60*60))

 lgb kfold: 1  of  5 : 
Training until validation scores don't improve for 100 rounds.
[10]	valid_0's multi_logloss: 1.02807
[20]	valid_0's multi_logloss: 0.967679
[30]	valid_0's multi_logloss: 0.91592
[40]	valid_0's multi_logloss: 0.870951
[50]	valid_0's multi_logloss: 0.831783
[60]	valid_0's multi_logloss: 0.797422
[70]	valid_0's multi_logloss: 0.76731
[80]	valid_0's multi_logloss: 0.740534
[90]	valid_0's multi_logloss: 0.716882
[100]	valid_0's multi_logloss: 0.695898
[110]	valid_0's multi_logloss: 0.677234
[120]	valid_0's multi_logloss: 0.660455
[130]	valid_0's multi_logloss: 0.645368
[140]	valid_0's multi_logloss: 0.631732
[150]	valid_0's multi_logloss: 0.619467
[160]	valid_0's multi_logloss: 0.608395
[170]	valid_0's multi_logloss: 0.598421
[180]	valid_0's multi_logloss: 0.589323
[190]	valid_0's multi_logloss: 0.581047
[200]	valid_0's multi_logloss: 0.573515
[210]	valid_0's multi_logloss: 0.566621
[220]	valid_0's multi_logloss: 0.560299
[230]	valid_0's multi_logloss: 0.554487
[240]

[460]	valid_0's multi_logloss: 0.493458
[470]	valid_0's multi_logloss: 0.492345
[480]	valid_0's multi_logloss: 0.491302
[490]	valid_0's multi_logloss: 0.490313
[500]	valid_0's multi_logloss: 0.4894
[510]	valid_0's multi_logloss: 0.488524
[520]	valid_0's multi_logloss: 0.487692
[530]	valid_0's multi_logloss: 0.486887
[540]	valid_0's multi_logloss: 0.486137
[550]	valid_0's multi_logloss: 0.485446
[560]	valid_0's multi_logloss: 0.484786
[570]	valid_0's multi_logloss: 0.484152
[580]	valid_0's multi_logloss: 0.48356
[590]	valid_0's multi_logloss: 0.483002
[600]	valid_0's multi_logloss: 0.482471
[610]	valid_0's multi_logloss: 0.481964
[620]	valid_0's multi_logloss: 0.481501
[630]	valid_0's multi_logloss: 0.481046
[640]	valid_0's multi_logloss: 0.480618
[650]	valid_0's multi_logloss: 0.480201
[660]	valid_0's multi_logloss: 0.479812
[670]	valid_0's multi_logloss: 0.479446
[680]	valid_0's multi_logloss: 0.4791
[690]	valid_0's multi_logloss: 0.478766
[700]	valid_0's multi_logloss: 0.478454
[710]

[800]	valid_0's multi_logloss: 0.477056
[810]	valid_0's multi_logloss: 0.476888
[820]	valid_0's multi_logloss: 0.476724
[830]	valid_0's multi_logloss: 0.476556
[840]	valid_0's multi_logloss: 0.476416
[850]	valid_0's multi_logloss: 0.476271
[860]	valid_0's multi_logloss: 0.476133
[870]	valid_0's multi_logloss: 0.476002
[880]	valid_0's multi_logloss: 0.475866
[890]	valid_0's multi_logloss: 0.47574
[900]	valid_0's multi_logloss: 0.475642
[910]	valid_0's multi_logloss: 0.475541
[920]	valid_0's multi_logloss: 0.475442
[930]	valid_0's multi_logloss: 0.475344
[940]	valid_0's multi_logloss: 0.475253
[950]	valid_0's multi_logloss: 0.475148
[960]	valid_0's multi_logloss: 0.475065
[970]	valid_0's multi_logloss: 0.474977
[980]	valid_0's multi_logloss: 0.474897
[990]	valid_0's multi_logloss: 0.474815
[1000]	valid_0's multi_logloss: 0.474739
[1010]	valid_0's multi_logloss: 0.47467
[1020]	valid_0's multi_logloss: 0.474609
[1030]	valid_0's multi_logloss: 0.47455
[1040]	valid_0's multi_logloss: 0.47449

[1080]	valid_0's multi_logloss: 0.474643
[1090]	valid_0's multi_logloss: 0.4746
[1100]	valid_0's multi_logloss: 0.474558
[1110]	valid_0's multi_logloss: 0.474518
[1120]	valid_0's multi_logloss: 0.474479
[1130]	valid_0's multi_logloss: 0.474448
[1140]	valid_0's multi_logloss: 0.474421
[1150]	valid_0's multi_logloss: 0.47439
[1160]	valid_0's multi_logloss: 0.474356
[1170]	valid_0's multi_logloss: 0.474334
[1180]	valid_0's multi_logloss: 0.474312
[1190]	valid_0's multi_logloss: 0.474287
[1200]	valid_0's multi_logloss: 0.474263
[1210]	valid_0's multi_logloss: 0.47424
[1220]	valid_0's multi_logloss: 0.47422
[1230]	valid_0's multi_logloss: 0.474205
[1240]	valid_0's multi_logloss: 0.474199
[1250]	valid_0's multi_logloss: 0.474171
[1260]	valid_0's multi_logloss: 0.474153
[1270]	valid_0's multi_logloss: 0.47413
[1280]	valid_0's multi_logloss: 0.474124
[1290]	valid_0's multi_logloss: 0.474117
[1300]	valid_0's multi_logloss: 0.474104
[1310]	valid_0's multi_logloss: 0.474103
[1320]	valid_0's multi

In [ ]:
s = np.zeros([test.shape[0]])
for i in tqdm(range(test.shape[0])) :
    s[i] = sub[i].argmax()

In [ ]:
submission=test_ID.to_frame()
submission['target'] = s
submission.to_csv('lgb_update2.csv', index=False, float_format='%.5f')